In [141]:
import pandas as pd
import numpy as np
import json

# Training Data Generation Example
An example of how to generate training data for the data description quality model fine-tuning.

In [ ]:
# This dataset is one of the datasets that is part of the NRT Commercial dataset. It contains technical descriptions and comments.
df = pd.read_csv('data/Commercial_NRT Technical Description.csv')
df.shape

(976, 10)

In [143]:
df.head()

,FilePath,FileName,FileAttributes,TechnicalDescription,L3 - Himanshu: 4-Mar,L3 Reviewer Comments,L3 Review Status,L3 Reviewer,1st march,4th march
0,UDPCommercial_NRT,vwdimarrservice,ARRServiceId,This Attribute has a static value - <3001> dec...,Ok,Provide the logic for surrogate key generation...,NaN,NaN,Updated,NaN
1,UDPCommercial_NRT,vwdimarrservice,ARRType,Direct mapping to ARRType column coming from s...,Ok,NaN,In desc source table is mentioned as DimARRSer...,NaN,Updated,NaN
2,UDPCommercial_NRT,vwdimarrservice,ServiceName,Direct mapping to ServiceName column coming fr...,Ok,NaN,ref rno - 3,NaN,Updated,NaN
3,UDPCommercial_NRT,vwdimarrservice,ServicePackageSKU,Direct mapping to ServicePackageSKU column com...,Ok,NaN,ref rno - 3,NaN,Updated,NaN
4,UDPCommercial_NRT,vwdimarrservicepackage,ARRServicePackageId,"This Attribute has a static value (2001,2002,2...",Ok,Provide the logic for surrogate key generation...,ref rno - 3,NaN,Updated,NaN


## Exploring the data
Looking at the data, there appear to be various review comments in the <code>L3 Review Status</code> column, as well as `L3 - Himanshu: 4-Mar` and `L3 Reviewer Comments` columns. We can use these columns to generate training data for the data description quality model, feeding them as input to a GenAI model to reverse engineer what the comment might have been before being updated to the final passing version.

In [144]:
df['L3 Review Status'].unique()

array([nan,
       'In desc source table is mentioned as DimARRService table  and then source is mentioned as static source. \nIs it a static source file? \nRe check and phrase it accordingly.',
       'ref rno - 3', 'Add source table and souce name', 'Ok',
       'Hardcoded 1 or 2 on what condition? For which value it is 1 or 2 add the details.\n\nSurrogateKey generated within UDP NRT to identify unique records of dimkpistate table. \nLogic: When KPIState = "State1" then hardcoded as 1 \nwhen KPIState = "State2" then hardcoded as 2  \n',
       'Hardcoded 1 or 2 on what condition? For which value it is 1 or 2 add the details',
       'Ref rno - 27', 'Add source table name',
       'Instead of ND mention it as Commercial NextDay Datamart within SA&DP / UDPCOmmercial NextDay Datamart',
       'THis is an indirect mapping to DimGeo.\nMention it accordingly and the join logic',
       'In join logic DimGeo is not used', 'Ref rno - 143', 'OK',
       'what is the significance of - referenc

In addition, comments like "ref rno - 3" mean to refer to another row in the dataset. We can use this information to find the corresponding row and use the review status from that row. In this case, it is assumed that rno - 3 is referring to the second row in the dataset, as the header row is row 1 in excel.

In [145]:
ref_rno_3 = df['L3 Review Status'].iloc[1]

### Fail rows

Rows that have comments that are not NaN or "ok" can be reverse engineered to generate training data. The rows that have a comment in "L3 - Himanshu: 4-Mar", "L3 Reviewer Comments", or "L3 Review Status" can be used to generate training data.

In [146]:
# Filter for rows where values in "L3 - Himanshu: 4-Mar", "L3 Reviewer Comments", "L3 Review Status"
# Are not "Ok" or null

columns_to_check = ["L3 - Himanshu: 4-Mar", "L3 Reviewer Comments", "L3 Review Status"]

# Function to check if a row should be kept
def valid_row(row):
    # Check if at least one column is NOT 'ok' or NaN
    has_meaningful_comment = any([x != 'Ok' and x != 'ok' and pd.notnull(x) for x in row])
    return has_meaningful_comment

# Apply the filtering function
fail_descriptions = df[df[columns_to_check].apply(valid_row, axis=1)]

# Display filtered DataFrame
fail_descriptions

,FilePath,FileName,FileAttributes,TechnicalDescription,L3 - Himanshu: 4-Mar,L3 Reviewer Comments,L3 Review Status,L3 Reviewer,1st march,4th march
0,UDPCommercial_NRT,vwdimarrservice,ARRServiceId,This Attribute has a static value - <3001> dec...,Ok,Provide the logic for surrogate key generation...,NaN,NaN,Updated,NaN
1,UDPCommercial_NRT,vwdimarrservice,ARRType,Direct mapping to ARRType column coming from s...,Ok,NaN,In desc source table is mentioned as DimARRSer...,NaN,Updated,NaN
2,UDPCommercial_NRT,vwdimarrservice,ServiceName,Direct mapping to ServiceName column coming fr...,Ok,NaN,ref rno - 3,NaN,Updated,NaN
3,UDPCommercial_NRT,vwdimarrservice,ServicePackageSKU,Direct mapping to ServicePackageSKU column com...,Ok,NaN,ref rno - 3,NaN,Updated,NaN
4,UDPCommercial_NRT,vwdimarrservicepackage,ARRServicePackageId,"This Attribute has a static value (2001,2002,2...",Ok,Provide the logic for surrogate key generation...,ref rno - 3,NaN,Updated,NaN
...,...,...,...,...,...,...,...,...,...,...
931,UDPCommercial_NRT,vwfactsrclosedevents,SRCreatedDateTimeSeconds,Derived attribute of datatype int based on Cre...,"Logic is not clear, check\nupdated\nok",NaN,NaN,NaN,NaN,Updated
936,UDPCommercial_NRT,vwfactsrclosedevents,SRKey,Surrogate key generated within UDP NRT to iden...,ok,Provide the logic for surrogate key generation...,NaN,Ram,NaN,Updated
940,UDPCommercial_NRT,vwfactsrclosedevents,SRModifiedTimeId,Derived by applying decode logic on EventDateT...,Logic update by referring to SRCreatedDateTime...,NaN,NaN,NaN,NaN,Updated
949,UDPCommercial_NRT,vwfactsrclosedevents,SRScopeTimeDateTimeId,Derived by applying decode logic on StateAnnot...,Logic update by referring to SRCreatedDateTime...,NaN,NaN,NaN,NaN,Updated


In [148]:
fail_descriptions

,FilePath,FileName,FileAttributes,TechnicalDescription,L3 - Himanshu: 4-Mar,L3 Reviewer Comments,L3 Review Status,L3 Reviewer,1st march,4th march
0,UDPCommercial_NRT,vwdimarrservice,ARRServiceId,This Attribute has a static value - <3001> dec...,Ok,Provide the logic for surrogate key generation...,NaN,NaN,Updated,NaN
1,UDPCommercial_NRT,vwdimarrservice,ARRType,Direct mapping to ARRType column coming from s...,Ok,NaN,In desc source table is mentioned as DimARRSer...,NaN,Updated,NaN
2,UDPCommercial_NRT,vwdimarrservice,ServiceName,Direct mapping to ServiceName column coming fr...,Ok,NaN,ref rno - 3,NaN,Updated,NaN
3,UDPCommercial_NRT,vwdimarrservice,ServicePackageSKU,Direct mapping to ServicePackageSKU column com...,Ok,NaN,ref rno - 3,NaN,Updated,NaN
4,UDPCommercial_NRT,vwdimarrservicepackage,ARRServicePackageId,"This Attribute has a static value (2001,2002,2...",Ok,Provide the logic for surrogate key generation...,ref rno - 3,NaN,Updated,NaN
...,...,...,...,...,...,...,...,...,...,...
931,UDPCommercial_NRT,vwfactsrclosedevents,SRCreatedDateTimeSeconds,Derived attribute of datatype int based on Cre...,"Logic is not clear, check\nupdated\nok",NaN,NaN,NaN,NaN,Updated
936,UDPCommercial_NRT,vwfactsrclosedevents,SRKey,Surrogate key generated within UDP NRT to iden...,ok,Provide the logic for surrogate key generation...,NaN,Ram,NaN,Updated
940,UDPCommercial_NRT,vwfactsrclosedevents,SRModifiedTimeId,Derived by applying decode logic on EventDateT...,Logic update by referring to SRCreatedDateTime...,NaN,NaN,NaN,NaN,Updated
949,UDPCommercial_NRT,vwfactsrclosedevents,SRScopeTimeDateTimeId,Derived by applying decode logic on StateAnnot...,Logic update by referring to SRCreatedDateTime...,NaN,NaN,NaN,NaN,Updated


In [149]:
# replace all 'ref rno - 3' with ref_rno_3
fail_descriptions['L3 Review Status'] = fail_descriptions['L3 Review Status'].replace('ref rno - 3', ref_rno_3)

/var/folders/58/jr_gdrxd4bn3hwrmrhn8x7mh0000gn/T/ipykernel_82072/1922106112.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fail_descriptions['L3 Review Status'] = fail_descriptions['L3 Review Status'].replace('ref rno - 3', ref_rno_3)


### Pass rows
Passing rows are rows that have been reviewed and approved. We can use these rows to generate training data for the data description quality model. We can check if all review comment columns are empty or "ok". This means that the row has been reviewed and approved. In addition, the final descriptions of "fail" rows can be used, as they have been reviewed and updated to pass.

In [147]:
# correct descriptions
# Function to check if a row should be kept
def pass_row(row):
    # Check if at least one column is NOT 'ok' or NaN
    has_meaningful_comment = not any([x != 'Ok' and x != 'ok' and pd.notnull(x) for x in row])
    return has_meaningful_comment

# Apply the filtering function
pass_descriptions = df[df[columns_to_check].apply(pass_row, axis=1)]

# Display filtered DataFrame
pass_descriptions

,FilePath,FileName,FileAttributes,TechnicalDescription,L3 - Himanshu: 4-Mar,L3 Reviewer Comments,L3 Review Status,L3 Reviewer,1st march,4th march
7,UDPCommercial_NRT,vwdimcausetype,CauseType,Direct mapping to CaseType column in Cases tab...,Ok,Ok,NaN,Ram,Updated,NaN
9,UDPCommercial_NRT,vwdimcausetype,CESBICreatedDateTime,Timestamp(UTC) at which record is inserted by ...,Ok,Ok,NaN,Ram,Updated,NaN
10,UDPCommercial_NRT,vwdimcausetype,CESBILastUpdatedDateTime,Timestamp(UTC) at which record is updated by E...,Ok,Ok,NaN,Ram,Updated,NaN
11,UDPCommercial_NRT,vwdimicmsrsnapshot,CreatedDateTime,Direct mapping to CreateDate column in Inciden...,Ok,Ok,NaN,Ram,Updated,NaN
12,UDPCommercial_NRT,vwdimicmsrsnapshot,ModifiedDateTime,Direct mapping to ModifiedDate column coming f...,Ok,Ok,NaN,Ram,Updated,NaN
...,...,...,...,...,...,...,...,...,...,...
971,UDPCommercial_NRT,vwfactsrclosedevents,TransferReason,Direct mapping to TransferReason column in Cas...,ok,NaN,NaN,NaN,NaN,Updated
972,UDPCommercial_NRT,vwfactsrclosedevents,TransferredBy,Direct mapping to UpdatedBy column in Cases t...,ok,NaN,NaN,NaN,NaN,Updated
973,UDPCommercial_NRT,vwfactsrclosedevents,TransferredDateTime,Direct mapping to EventDateTime column in Case...,ok,NaN,NaN,NaN,NaN,Updated
974,UDPCommercial_NRT,vwfactsrclosedevents,TransferredFrom,Direct mapping to TransferredFrom column in Ca...,ok,NaN,NaN,NaN,NaN,Updated


We will use the fail descriptions along with their comments to generate training data for the model by passing in the comment and prompting it to generate what the description might have been before the comment was added. We will also use the pass descriptions to generate training data for the model by passing in the final description and prompting it to generate why this description passed.

### Prompt:
Reverse engineer what the original technical description looked like before being updated according the review comments. 
Output only a list of the predicted original technical descriptions, the reviewer comment, and the corresponding updated description (the description from the input file)
With this json input, reverse engineer the descriptions based on what it looked like before the comments were made. Update the comment to look like the reasoning output from an LLM. Output a new json object with format:
{"reverse_engineered_description": <reverse engineered description>, "comment": <updated comment(s)>, "final_description": <original description>} for each description

In [164]:
#output technical description, review columns to json for easier processing
fail_descriptions[['TechnicalDescription', 'L3 - Himanshu: 4-Mar', 'L3 Reviewer Comments', 'L3 Review Status']].to_json('data/comments.json', orient='records')

In [152]:
# create new test dataframe with columns "description" and "pass"
temp_df1 = pd.DataFrame(columns=['description', 'pass', 'reason'])

In [153]:
# include all technical descriptions as "description" and "pass" as 1
temp_df1['description'] = pass_descriptions['TechnicalDescription']
temp_df1['pass'] = 1
temp_df1['reason'] = "N/A"

In [154]:
temp_df1.shape

(863, 3)

After generating the reverse engineered descriptions, we can build a dataset with the following columns:
- description
- pass (1 if the description passed, 0 if it failed)
- reason (the reason why the description passed or failed)

In [155]:
# read in reverse_engineered.json
# Make all ReverseEngineeredDescriptions as "description" and "pass" as 0

with open('data/reverse_engineered.json') as f:
    data = json.load(f)
    temp_df2 = pd.DataFrame(data)

# reformat the columns to match test_df
temp_df2['pass'] = 0
temp_df2 = temp_df2.rename(columns={"reverse_engineered_description": "description"})
temp_df2['reason'] = temp_df2['comment']
temp_df2 = temp_df2.drop(columns=['comment'])
temp_df2 = temp_df2.drop(columns=['final_description'])
temp_df2.head()

,description,pass,reason
0,This Attribute has a static value - <3001> upl...,0,Provide the logic for surrogate key generation...
1,Direct mapping to ARRType column coming from D...,0,Is it a static source file? Re-check and phras...
2,Direct mapping to ServiceName column coming fr...,0,Is it a static source file? Re-check and phras...
3,Direct mapping to ServicePackageSKU column com...,0,Is it a static source file? Re-check and phras...
4,"This Attribute has a static value (2001,2002,2...",0,Provide the logic for surrogate key generation...


In [156]:
# combine the two dataframes
desc_df = pd.concat([temp_df1, temp_df2], ignore_index=True)
print(desc_df.shape)
desc_df.tail()

(951, 3)


,description,pass,reason
946,Derived attribute of datatype int based on Cre...,0,"Logic is not clear, check"
947,Surrogate key generated within UDP NRT to iden...,0,Provide the logic for surrogate key generation...
948,Derived by applying decode logic on EventDateT...,0,Logic update by referring to SRCreatedDateTime...
949,Derived by applying decode logic on StateAnnot...,0,Logic update by referring to SRCreatedDateTime...
950,Derived by applying decode logic on IncidentTy...,0,Write the logic in steps


In [157]:
desc_df['pass'].value_counts()

pass
1    863
0     88
Name: count, dtype: int64

In [158]:
# hold out 10% of the data for testing
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(desc_df, test_size=0.1)

train_df.value_counts("pass")


pass
1    776
0     79
Name: count, dtype: int64

In [159]:
test_df.value_counts("pass")

pass
1    87
0     9
Name: count, dtype: int64

In [ ]:
# format data OpenAI GPT3.5 fine-tuning in JSONL format
# format: {"messages": [{"role": "system", "content": <system prompt>}, {"role": "user", "content": <description>}, {"role": "assistant", "content": <pass or fail> <reason>}]}
system_prompt = """You are an AI text quality reviewer tool. Your task is to review technical descriptions for data entities and attributes. Based on the quality of the description, you will output either 'Pass' or 'Fail'.
'Pass' means the description meets the quality standards and provides clear, accurate, and complete information.
'Fail' means the description does not meet the required standards, and you must provide a specific reason for the failure.
Your output format will be: <Pass or Fail>, <N/A or reason for failure>"""

with open('data/train.jsonl', 'w') as f:
    for i, row in train_df.iterrows():
        content = {"messages": [{"role": "system", "content": system_prompt}, {"role": "user", "content": row['description']}, {"role": "assistant", "content": f"{"pass," if row['pass'] else "fail,"} {row['reason']}"}]}
        json.dump(content, f)
        f.write('\n')

with open('data/test.jsonl', 'w') as f:
    for i, row in test_df.iterrows():
        content = {"messages": [{"role": "system", "content": system_prompt}, {"role": "user", "content": row['description']}, {"role": "assistant", "content": f"{"pass," if row['pass'] else "fail,"} {row['reason']}"}]}
        json.dump(content, f)
        f.write('\n')

In [163]:
# format data in Alpaca format
# format: {"instruction": <system prompt>, "input": <description>, "output": <pass or fail> <reason>}
# write as json file [{}, {}, ...]

with open('data/train_alpaca.json', 'w') as f:
    content_list = []
    for i, row in train_df.iterrows():
        content = {"instruction": system_prompt, "input": row['description'], "output": f"{"pass," if row['pass'] else "fail,"} {row['reason']}"}
        content_list.append(content)
    json.dump(content_list, f)
    f.write('\n')